In [1]:
## 加载2023年的mean

import pandas as pd

uav_2023_path = r'D:\work\DATA\DATA_TS4GPC\raw\UAV\CXZ-WN\CXZ-WN-2024'
bands_list = ['Blue','Green','NIR','Red','RedEdge']

# 获取这个文件夹中所有multiband.csv文件的完整路径
import os

def get_csv_file_paths(folder_path):
    file_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('multiband.csv'):
                file_paths.append(os.path.join(root, file))
    return file_paths

# 请将此处的路径替换为你实际的文件夹路径 
file_paths = get_csv_file_paths(uav_2023_path)

def calculate_vegetation_indices(R, G, B, RE=None, NIR=None, L1=1, L2=0.5):
    # 归一化红光指数
    r = R / (R + G + B)
    
    # 归一化绿光指数
    g = G / (R + G + B)
    
    # 归一化蓝光指数
    b = B / (R + G + B)
    
    # 绿蓝比指数GBRI
    GBRI = b / g
    
    # 红绿比指数RGRI
    RGRI = r / g
    
    # 超蓝指数ExB
    ExB = 1.4 * b - g
    
    # 超绿指数ExG
    ExG = 2 * g - r - b
    
    # 超红指数ExR
    ExR = 1.4 * r - g
    
    # 超绿超红差分指数ExGR
    ExGR = ExG - ExR
    
    # 植被颜色提取指数CIVE
    CIVE = 0.441 * r - 0.881 * g + 0.385 * b + 18.78745
    
    # 植被指数VEG 
    # VEG = g / (r**a)*b**(1-a)
    VEG = NIR/(R + G + B)**0.5
    
    # 改进型绿红植被指数MGRVI
    MGRVI = (G**2 - R**2) / (G**2 + R**2)
    
    # 归一化绿红差分指数NGRDI
    NGRDI = (G - R) / (G + R)
    
    # 可见光波段差异植被指数VDVI
    VDVI = (2 * g - r - b) / (2 * g + r + b)
    
    # 综合指数1 COMI
    # COMI = 0.25 * ExG + 0.3 * ExGR + 0.33 * CIVE + 0.12 * VEG
    
    # 综合指数2 COM2
    # COM2 = 0.36 * ExG + 0.47 * CIVE + 0.17 * VEG
    
    # 耐大气植被指数ARVI
    ARVI = (NIR - (2 * R - B)) / (NIR + (2 * R - B))
    
    # 差值植被指数DVI
    DVI = NIR - R
    
    # 增强植被指数EVI
    EVI = 2.5 * (NIR - R) / (NIR + 6 * R - 7.5 * B + L1)
    
    # 绿色归一化差异植被指数GNDVI
    GNDVI = (NIR - G) / (NIR + G)
    
    # 归一化差异红边植被指数NDRE (Only if RE is provided)
    NDRE = (NIR - RE) / (NIR + RE)
    
    # 归一化差异植被指数NDVI
    NDVI = (NIR - R) / (NIR + R)
    
    # 优化的土壤调节植被指数OSAVI
    OSAVI = (NIR - R) / (NIR + R + 0.16)
    
    # 比值植被指数RVI
    RVI = NIR / R
    
    # 土壤调节植被指数SAVI
    # SAVI = ((NIR - R) / (NIR + R + L1)) * (1 + L2)
    
    # 叶面叶绿素指数LCI
    LCI = (NIR - R) / (NIR + R)
    
    # 
    GVI = NIR / G 

    # 
    NVI = NIR / (NIR + R)

    # 
    TVI = 0.5 * (120 * (NIR - G) - 200 * (R - G))

    # 可见光大气阻抗植被指数 VARI
    VARI = (G - R) / (G + R - B)
    
    # 短波红外增强型归一化植被指数 ENDVI
    ENDVI = (NIR - G - 2 * B) / (NIR + G + 2 * B)
    
    # 叶化学反射率指数 DATT
    DATT = (NIR - RE) / (NIR - R)

    # 归一化水指数 NDWI
    NDWI = (NIR - G) / (NIR + G)

    # 水指数
    WI = NIR / G

    return {
        'r': r, 
        'g': g, 
        'b': b, 
        'GBRI': GBRI, 
        'RGRI': RGRI, 
        'ExB': ExB, 
        'ExG': ExG,
        'ExR': ExR, 
        'ExGR': ExGR, 
        'CIVE': CIVE, 
        'MGRVI': MGRVI,
        'NGRDI': NGRDI, 
        'VDVI': VDVI, 
        'ARVI': ARVI,
        'DVI': DVI, 
        'EVI': EVI, 
        'GNDVI': GNDVI, 
        'NDRE': NDRE, 
        'NDVI': NDVI,
        'OSAVI': OSAVI, 
        'RVI': RVI, 
        # 'SAVI': SAVI, 
        'LCI': LCI,
        'VEG':VEG,
        'VARI': VARI,
        'ENDVI': ENDVI,
        'DATT': DATT,
        'GVI': GVI,
        'NVI': NVI,
        'TVI': TVI,
        'NDWI': NDWI,
        'WI': WI
    }

out_dir = r'C:\Users\tiger\Desktop\VI'

import re



for path in file_paths:
    print('正在处理：',path)
    df = pd.read_csv(path)
    B,G,R,RE,NIR = df.iloc[:,1],df.iloc[:,2],df.iloc[:,3],df.iloc[:,4],df.iloc[:,5] 

    dir_vi = calculate_vegetation_indices(R, G, B, RE, NIR)
    df_vi = pd.DataFrame(dir_vi)

    match = re.search(r'([^\\]+)\\multiband\.csv$', path)
    if match:
        out_path = os.path.join(out_dir, f'{match.group(1)}_vi.csv')
        df_vi.to_csv(out_path, index=False)
        print('保存成功：',out_path)
    else:
        print('无法匹配文件名：', path)


正在处理： D:\work\DATA\DATA_TS4GPC\raw\UAV\CXZ-WN\CXZ-WN-2024\CXZ-WN-240130\multiband.csv
保存成功： C:\Users\tiger\Desktop\VI\CXZ-WN-240130_vi.csv
正在处理： D:\work\DATA\DATA_TS4GPC\raw\UAV\CXZ-WN\CXZ-WN-2024\CXZ-WN-240312\multiband.csv
保存成功： C:\Users\tiger\Desktop\VI\CXZ-WN-240312_vi.csv
正在处理： D:\work\DATA\DATA_TS4GPC\raw\UAV\CXZ-WN\CXZ-WN-2024\CXZ-WN-240401\multiband.csv
保存成功： C:\Users\tiger\Desktop\VI\CXZ-WN-240401_vi.csv
正在处理： D:\work\DATA\DATA_TS4GPC\raw\UAV\CXZ-WN\CXZ-WN-2024\CXZ-WN-240409\multiband.csv
保存成功： C:\Users\tiger\Desktop\VI\CXZ-WN-240409_vi.csv
正在处理： D:\work\DATA\DATA_TS4GPC\raw\UAV\CXZ-WN\CXZ-WN-2024\CXZ-WN-240412\multiband.csv
保存成功： C:\Users\tiger\Desktop\VI\CXZ-WN-240412_vi.csv
正在处理： D:\work\DATA\DATA_TS4GPC\raw\UAV\CXZ-WN\CXZ-WN-2024\CXZ-WN-240417\multiband.csv
保存成功： C:\Users\tiger\Desktop\VI\CXZ-WN-240417_vi.csv
正在处理： D:\work\DATA\DATA_TS4GPC\raw\UAV\CXZ-WN\CXZ-WN-2024\CXZ-WN-240428\multiband.csv
保存成功： C:\Users\tiger\Desktop\VI\CXZ-WN-240428_vi.csv
正在处理： D:\work\DATA\DATA_TS4

In [6]:
df = pd.read_csv(file_paths[0])
B,G,R,RE,NIR = df.iloc[:,1],df.iloc[:,2],df.iloc[:,3],df.iloc[:,4],df.iloc[:,5]
B

0      0.053308
1      0.052574
2      0.059001
3      0.057293
4      0.055721
         ...   
235    0.061005
236    0.063256
237    0.061548
238    0.054061
239    0.054621
Name: Blue, Length: 240, dtype: float64

In [7]:
def calculate_vegetation_indices(R, G, B, RE=None, NIR=None, L1=1, L2=0.5):
    # 归一化红光指数
    r = R / (R + G + B)
    
    # 归一化绿光指数
    g = G / (R + G + B)
    
    # 归一化蓝光指数
    b = B / (R + G + B)
    
    # 绿蓝比指数GBRI
    GBRI = b / g
    
    # 红绿比指数RGRI
    RGRI = r / g
    
    # 超蓝指数ExB
    ExB = 1.4 * b - g
    
    # 超绿指数ExG
    ExG = 2 * g - r - b
    
    # 超红指数ExR
    ExR = 1.4 * r - g
    
    # 超绿超红差分指数ExGR
    ExGR = ExG - ExR
    
    # 植被颜色提取指数CIVE
    CIVE = 0.441 * r - 0.881 * g + 0.385 * b + 18.78745
    
    # 植被指数VEG 
    # VEG = g / (r**a)*b**(1-a)
    VEG = NIR/(R + G + B)**0.5
    
    # 改进型绿红植被指数MGRVI
    MGRVI = (G**2 - R**2) / (G**2 + R**2)
    
    # 归一化绿红差分指数NGRDI
    NGRDI = (G - R) / (G + R)
    
    # 可见光波段差异植被指数VDVI
    VDVI = (2 * g - r - b) / (2 * g + r + b)
    
    # 综合指数1 COMI
    # COMI = 0.25 * ExG + 0.3 * ExGR + 0.33 * CIVE + 0.12 * VEG
    
    # 综合指数2 COM2
    # COM2 = 0.36 * ExG + 0.47 * CIVE + 0.17 * VEG
    
    # 耐大气植被指数ARVI
    ARVI = (NIR - (2 * R - B)) / (NIR + (2 * R - B))
    
    # 差值植被指数DVI
    DVI = NIR - R
    
    # 增强植被指数EVI
    EVI = 2.5 * (NIR - R) / (NIR + 6 * R - 7.5 * B + L1)
    
    # 绿色归一化差异植被指数GNDVI
    GNDVI = (NIR - G) / (NIR + G)
    
    # 归一化差异红边植被指数NDRE (Only if RE is provided)
    NDRE = (NIR - RE) / (NIR + RE)
    
    # 归一化差异植被指数NDVI
    NDVI = (NIR - R) / (NIR + R)
    
    # 优化的土壤调节植被指数OSAVI
    OSAVI = (NIR - R) / (NIR + R + 0.16)
    
    # 比值植被指数RVI
    RVI = NIR / R
    
    # 土壤调节植被指数SAVI
    # SAVI = ((NIR - R) / (NIR + R + L1)) * (1 + L2)
    
    # 叶面叶绿素指数LCI
    LCI = (NIR - R) / (NIR + R)
    
    return {
        'r': r, 
        'g': g, 
        'b': b, 
        'GBRI': GBRI, 
        'RGRI': RGRI, 
        'ExB': ExB, 
        'ExG': ExG,
        'ExR': ExR, 
        'ExGR': ExGR, 
        'CIVE': CIVE, 
        'MGRVI': MGRVI,
        'NGRDI': NGRDI, 
        'VDVI': VDVI, 
        'ARVI': ARVI,
        'DVI': DVI, 
        'EVI': EVI, 
        'GNDVI': GNDVI, 
        'NDRE': NDRE, 
        'NDVI': NDVI,
        'OSAVI': OSAVI, 
        'RVI': RVI, 
        # 'SAVI': SAVI, 
        'LCI': LCI,
        'VEG':VEG
    }
 

In [ ]:
dir_vi = calculate_vegetation_indices(R, G, B, RE, NIR)
df_vi = pd.DataFrame(dir_vi)

# df_vi.to_csv(r'D:\work\官村无人机影像\关村0718\VI.csv', index=False)

,r,g,b,GBRI,RGRI,ExB,ExG,ExR,ExGR,CIVE,...,ARVI,DVI,EVI,GNDVI,NDRE,NDVI,OSAVI,RVI,LCI,VEG
0,0.615537,0.237593,0.146870,0.618157,2.590716,-0.031975,-0.287220,0.624158,-0.911378,18.906127,...,-0.382947,-0.047831,-0.056504,0.341255,0.293063,-0.119876,-0.085564,0.785912,-0.119876,0.291447
1,0.620346,0.236222,0.143431,0.607188,2.626110,-0.035419,-0.291333,0.632262,-0.923595,18.908132,...,-0.384115,-0.048421,-0.056331,0.347872,0.305654,-0.119162,-0.085497,0.787052,-0.119162,0.295597
2,0.569611,0.263954,0.166434,0.630543,2.157995,-0.030946,-0.208138,0.533502,-0.741640,18.870182,...,-0.343734,-0.033504,-0.043232,0.285698,0.186365,-0.090466,-0.063174,0.834078,-0.090466,0.282874
3,0.584948,0.255111,0.159942,0.626950,2.292920,-0.031192,-0.234668,0.563817,-0.798485,18.882237,...,-0.356770,-0.038021,-0.047549,0.304799,0.223037,-0.099778,-0.070272,0.818549,-0.099778,0.286572
4,0.602609,0.244221,0.153170,0.627179,2.467477,-0.029783,-0.267338,0.599432,-0.866770,18.897013,...,-0.376304,-0.045785,-0.055272,0.322526,0.254832,-0.116601,-0.082844,0.791150,-0.116601,0.287553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,0.573986,0.259759,0.166254,0.640031,2.209685,-0.027004,-0.220722,0.543822,-0.764543,18.875738,...,-0.349777,-0.037085,-0.046832,0.290933,0.187094,-0.096538,-0.068153,0.823921,-0.096538,0.286473
236,0.567061,0.263350,0.169589,0.643967,2.153254,-0.025926,-0.209949,0.530534,-0.740483,18.870804,...,-0.343737,-0.035806,-0.045431,0.282829,0.172216,-0.092471,-0.065433,0.830713,-0.092471,0.287695
237,0.572751,0.259139,0.168110,0.648724,2.210204,-0.023785,-0.222582,0.542712,-0.765294,18.876454,...,-0.351753,-0.038088,-0.048380,0.287939,0.182324,-0.099890,-0.070364,0.818363,-0.099890,0.283610
238,0.646315,0.216479,0.137206,0.633806,2.985576,-0.024391,-0.350562,0.688362,-1.038924,18.934581,...,-0.416352,-0.067057,-0.072570,0.374884,0.334708,-0.151625,-0.111343,0.736677,-0.151625,0.298865


In [13]:
out_dir = r'C:\Users\tiger\Desktop\VI'


In [11]:
import re

match = re.search(r'([^\\]+)\\multiband\.csv$', file_paths[0])
if match:
    print(match.group(1))

CXZ-WN-230304


In [16]:
out_path = os.path.join(out_dir, f'{match.group(1)}_vi.csv')
out_path

'C:\\Users\\tiger\\Desktop\\VI\\CXZ-WN-230304_vi.csv'

In [17]:
df_vi.to_csv(out_path, index=False)